# Atualização do Dataset e Uso para Validação do Modelo

## Status do Dataset

O dataset utilizado em nosso projeto foi **atualizado até a luta mais recente**. Agora, utilizaremos esses dados para prever os resultados das próximas lutas, permitindo uma avaliação mais precisa da eficácia do modelo.


In [856]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

Primeiramente, realizaremos a leitura dos dados das 13 lutas que serão analisadas. Para garantir que o modelo possa utilizá-los corretamente, aplicaremos o mesmo processo de tratamento e pré-processamento utilizado nos dados de treinamento, assegurando consistência e compatibilidade entre as amostras.

In [857]:
test_fights = pd.read_csv('data/upcoming.csv')
training_data = pd.read_csv('data/ufc-master.csv')

data = pd.concat([test_fights, training_data])

In [858]:
test_fights.head(7)

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,TitleBout,WeightClass,Gender,NumberOfRounds,BlueCurrentLoseStreak,BlueCurrentWinStreak,BlueDraws,BlueAvgSigStrLanded,BlueAvgSigStrPct,BlueAvgSubAtt,BlueAvgTDLanded,BlueAvgTDPct,BlueLongestWinStreak,BlueLosses,BlueTotalRoundsFought,BlueTotalTitleBouts,BlueWinsByDecisionMajority,BlueWinsByDecisionSplit,BlueWinsByDecisionUnanimous,BlueWinsByKO,BlueWinsBySubmission,BlueWinsByTKODoctorStoppage,BlueWins,BlueStance,BlueHeightCms,BlueReachCms,BlueWeightLbs,RedCurrentLoseStreak,RedCurrentWinStreak,RedDraws,RedAvgSigStrLanded,RedAvgSigStrPct,RedAvgSubAtt,RedAvgTDLanded,RedAvgTDPct,RedLongestWinStreak,RedLosses,RedTotalRoundsFought,RedTotalTitleBouts,RedWinsByDecisionMajority,RedWinsByDecisionSplit,RedWinsByDecisionUnanimous,RedWinsByKO,RedWinsBySubmission,RedWinsByTKODoctorStoppage,RedWins,RedStance,RedHeightCms,RedReachCms,RedWeightLbs,RedAge,BlueAge,LoseStreakDif,WinStreakDif,LongestWinStreakDif,WinDif,LossDif,TotalRoundDif,TotalTitleBoutDif,KODif,SubDif,HeightDif,ReachDif,AgeDif,SigStrDif,AvgSubAttDif,AvgTDDif,EmptyArena,BMatchWCRank,RMatchWCRank,RWFlyweightRank,RWFeatherweightRank,RWStrawweightRank,RWBantamweightRank,RHeavyweightRank,RLightHeavyweightRank,RMiddleweightRank,RWelterweightRank,RLightweightRank,RFeatherweightRank,RBantamweightRank,RFlyweightRank,RPFPRank,BWFlyweightRank,BWFeatherweightRank,BWStrawweightRank,BWBantamweightRank,BHeavyweightRank,BLightHeavyweightRank,BMiddleweightRank,BWelterweightRank,BLightweightRank,BFeatherweightRank,BBantamweightRank,BFlyweightRank,BPFPRank,BetterRank,Finish,FinishDetails,FinishRound,FinishRoundTime,TotalFightTimeSecs,RedDecOdds,BlueDecOdds,RSubOdds,BSubOdds,RKOOdds,BKOOdds
0,Marvin Vettori,Roman Dolidze,NaN,NaN,NaN,NaN,2025-03-15,"Las Vegas, Nevada, USA",USA,NaN,False,Middleweight,MALE,5,0,2,0,3.16,0.42,1.1,1.33,0.42,4,3,26,0,0,1,2,5,0,0,8,Orthodox,187.96,193.04,185,1,0,1,4.56,0.45,0.5,1.66,0.45,5,5,51,1,0,0,7,0,2,0,9,Southpaw,182.88,187.96,185,31,36,-1,2,-1,-1,-2,-25,-1,5,-2,5.08,5.08,5,-1.40,0.6,-0.33,1,12.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,Red,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Chidi Njokuani,Elizeu Zaleski dos Santos,NaN,NaN,NaN,NaN,2025-03-15,"Las Vegas, Nevada, USA",USA,NaN,False,Welterweight,MALE,3,0,1,1,4.33,0.41,0.5,0.70,0.20,7,4,41,0,0,2,4,4,1,0,11,Orthodox,180.34,185.42,170,0,2,0,4.59,0.63,0.2,0.00,0.00,3,3,17,0,0,1,1,3,0,0,5,Orthodox,190.50,203.20,170,36,38,0,-1,4,6,1,24,0,1,1,-10.16,-17.78,2,-0.26,0.3,0.70,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alexander Hernandez,Kurt Holobaugh,NaN,NaN,NaN,NaN,2025-03-15,"Las Vegas, Nevada, USA",USA,NaN,False,Lightweight,MALE,3,0,1,0,4.63,0.45,0.8,0.76,0.33,1,6,22,0,0,0,1,0,1,0,2,Orthodox,180.34,177.80,155,0,1,0,4.43,0.41,0.1,0.95,0.29,2,7,32,0,0,1,3,3,0,0,7,Orthodox,175.26,182.88,155,32,38,0,0,-1,-5,-1,-10,0,-3,1,5.08,-5.08,6,0.20,0.7,-0.19,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Da'Mon Blackshear,Cody Gibson,NaN,NaN,NaN,NaN,2025-03-15,"Las Vegas, Nevada, USA",USA,NaN,False,Bantamweight,MALE,3,0,2,0,3.47,0.44,0.7,2.18,0.40,2,5,19,0,0,0,1,1,1,0,3,Orthodox,177.80,180.34,135,0,1,1,4.30,0.47,1.4,2.17,0.37,2,3,14,0,0,0,0,1,2,0,3,Switch,177.80,182.88,135,30,37,0,1,0,0,2,5,0,0,-1,0.00,-2.54,7,-0.83,-0.7,0.01,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Diyar Nurgozhay,Brendson Ribeiro,NaN,NaN,NaN,NaN,2025-03-15,"Las Vegas, Nevada, USA",USA,NaN,False,Light Heavyweight,MALE,3,0,1,0,3.16,0.42,0.4,1.27,0.50,1,2,8,0,0,1,0,1,0,0,2,Orthodox,190.50,205.74,205,0,1,0,3.52,0.52,0.0,0.00,0.00,

In [859]:
data = data.drop(columns=[
    'RedExpectedValue', 
    'BlueExpectedValue', 
    'EmptyArena', 
    'BMatchWCRank', 
    'RMatchWCRank',
    'RWFlyweightRank',
    'RWFeatherweightRank',
    'RWStrawweightRank',
    'RWBantamweightRank',
    'RHeavyweightRank',
    'RLightHeavyweightRank',
    'RMiddleweightRank',
    'RWelterweightRank',
    'RLightweightRank',
    'RFeatherweightRank',
    'RBantamweightRank',
    'RFlyweightRank',
    'RPFPRank',
    'BWFlyweightRank',
    'BWFeatherweightRank',
    'BWStrawweightRank',
    'BWBantamweightRank',
    'BHeavyweightRank',
    'BLightHeavyweightRank',
    'BMiddleweightRank',
    'BWelterweightRank',
    'BLightweightRank',
    'BFeatherweightRank',
    'BBantamweightRank',
    'BFlyweightRank',
    'BPFPRank',
    'FinishDetails',
    'FinishRoundTime',
    'RedDecOdds',
    'BlueDecOdds',
    'RSubOdds',
    'BSubOdds',
    'RKOOdds',
    'BKOOdds',
    'TotalFightTimeSecs',
    'Finish',
    'FinishRound',
    'Location',
    'Country',
    'BlueOdds',
    'RedOdds'
])




In [860]:
global_red_sig_str_mean = data['RedAvgSigStrLanded'].mean()
global_blue_sig_str_mean = data['BlueAvgSigStrLanded'].mean()

data['RedAvgSigStrLanded'] = (
    data['RedAvgSigStrLanded']
    .fillna(data.groupby('RedFighter')['RedAvgSigStrLanded'].transform('mean'))
    .fillna(global_red_sig_str_mean)
)

data['BlueAvgSigStrLanded'] = (
    data['BlueAvgSigStrLanded']
    .fillna(data.groupby('BlueFighter')['BlueAvgSigStrLanded'].transform('mean'))
    .fillna(global_blue_sig_str_mean)
)

In [861]:
global_red_sig_str_pctmean = data['RedAvgSigStrPct'].mean()
global_blue_sig_str_pctmean = data['BlueAvgSigStrPct'].mean()

data['RedAvgSigStrPct'] = (
    data['RedAvgSigStrPct']
    .fillna(data.groupby('RedFighter')['RedAvgSigStrPct'].transform('mean'))
    .fillna(global_red_sig_str_pctmean)
)

data['BlueAvgSigStrPct'] = (
    data['BlueAvgSigStrPct']
    .fillna(data.groupby('BlueFighter')['BlueAvgSigStrPct'].transform('mean'))
    .fillna(global_blue_sig_str_pctmean)
)

In [862]:
global_red_sub_att_mean = data['RedAvgSubAtt'].mean()
global_blue_sub_att_mean = data['BlueAvgSubAtt'].mean()

data['RedAvgSubAtt'] = (
    data['RedAvgSubAtt']
    .fillna(data.groupby('RedFighter')['RedAvgSubAtt'].transform('mean'))
    .fillna(global_red_sub_att_mean)
)

data['BlueAvgSubAtt'] = (
    data['BlueAvgSubAtt']
    .fillna(data.groupby('BlueFighter')['BlueAvgSubAtt'].transform('mean'))
    .fillna(global_blue_sub_att_mean)
)

In [863]:
global_red_td_mean = data['RedAvgTDLanded'].mean()
global_blue_td_mean = data['BlueAvgTDLanded'].mean()

data['RedAvgTDLanded'] = (
    data['RedAvgTDLanded']
    .fillna(data.groupby('RedFighter')['RedAvgTDLanded'].transform('mean'))
    .fillna(global_red_td_mean)
)

data['BlueAvgTDLanded'] = (
    data['BlueAvgTDLanded']
    .fillna(data.groupby('BlueFighter')['BlueAvgTDLanded'].transform('mean'))
    .fillna(global_blue_td_mean)
)

In [864]:
global_red_td_pct_mean = data['RedAvgTDPct'].mean()
global_blue_td_pct_mean = data['BlueAvgTDPct'].mean()

data['RedAvgTDPct'] = (
    data['RedAvgTDPct']
    .fillna(data.groupby('RedFighter')['RedAvgTDPct'].transform('mean'))
    .fillna(global_red_td_pct_mean)
)

data['BlueAvgTDPct'] = (
    data['BlueAvgTDPct']
    .fillna(data.groupby('BlueFighter')['BlueAvgTDPct'].transform('mean'))
    .fillna(global_blue_td_pct_mean)
)

In [865]:
global_blue_stance_mode = data['BlueStance'].mode()[0]  

data['BlueStance'] = (
    data['BlueStance']
    .fillna(data.groupby('BlueFighter')['BlueStance'].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan))
    .fillna(global_blue_stance_mode)
)


Separando as lutas de teste

In [866]:
test_fights= data.merge(
    test_fights[['RedFighter', 'BlueFighter', 'Date']],
    on=['RedFighter', 'BlueFighter','Date'],
    how='inner'
)

In [867]:
merged = data.merge(
    test_fights[['RedFighter', 'BlueFighter', 'Date']],
    on=['RedFighter', 'BlueFighter', 'Date'],
    how='left',
    indicator=True
)

data = merged[merged['_merge'] == 'left_only']

Calculo do elo para as amostras de treino

In [868]:
data = data.sort_values(by='Date').reset_index(drop=True)

In [869]:
def update_elo(winner_elo, loser_elo, K=150):
    """
    Atualiza o Elo após uma luta.
    K = Fator de peso (quanto maior, mais rápido a pontuação muda).
    """
    expected_win = 1 / (1 + 10 ** ((loser_elo - winner_elo) / 400))
    change = K * (1 - expected_win)  
    
    new_winner_elo = winner_elo + change
    new_loser_elo = loser_elo - change
    
    return new_winner_elo, new_loser_elo

In [870]:
elo_dict = {}
initial_elo = 1000

all_fighters = pd.concat([data['RedFighter'], data['BlueFighter']]).unique()

# Inicializar o Elo para todos os lutadores
for fighter in all_fighters:
    elo_dict[fighter] = initial_elo

# Função para atualizar o Elo
def update_elo(winner_elo, loser_elo, K=50):
    expected_win = 1 / (1 + 10 ** ((loser_elo - winner_elo) / 400))
    change = K * (1 - expected_win)
    return winner_elo + change, loser_elo - change

red_elo_current = []
blue_elo_current = []

for idx, row in data.iterrows():
    red = row['RedFighter']
    blue = row['BlueFighter']
    winner = row['Winner']
    
    # Elo atual antes da luta (armazenado no dicionário)
    red_elo = elo_dict[red]
    blue_elo = elo_dict[blue]
    
    # Atualizar Elo com base no resultado
    if winner == 'Red':
        new_red, new_blue = update_elo(red_elo, blue_elo)
    elif winner == 'Blue':
        new_blue, new_red = update_elo(blue_elo, red_elo)
    else:
        new_red = red_elo
        new_blue = blue_elo
    
    elo_dict[red] = new_red
    elo_dict[blue] = new_blue
    
    red_elo_current.append(new_red)
    blue_elo_current.append(new_blue)

data['RedEloCurrent'] = red_elo_current
data['BlueEloCurrent'] = blue_elo_current

In [871]:
import numpy as np

data['EloDiff'] = np.abs(data['RedEloCurrent'] - data['BlueEloCurrent'])

Atribui o elo as lutas teste

In [872]:
import pandas as pd

red_fighters = data[['RedFighter', 'RedEloCurrent', 'Date']].rename(
    columns={'RedFighter': 'Fighter', 'RedEloCurrent': 'EloCurrent'}
)
blue_fighters = data[['BlueFighter', 'BlueEloCurrent', 'Date']].rename(
    columns={'BlueFighter': 'Fighter', 'BlueEloCurrent': 'EloCurrent'}
)

all_fighters = pd.concat([red_fighters, blue_fighters], ignore_index=True)

all_fighters['Date'] = pd.to_datetime(all_fighters['Date'])  
latest_elos = (
    all_fighters
    .sort_values('Date')  
    .groupby('Fighter')    
    .tail(1)               
    .set_index('Fighter')['EloCurrent']
    .to_dict()
)

test_fights['RedEloCurrent'] = test_fights['RedFighter'].map(latest_elos)
test_fights['BlueEloCurrent'] = test_fights['BlueFighter'].map(latest_elos)


test_fights['RedEloCurrent'] = test_fights['RedEloCurrent'].fillna(initial_elo)
test_fights['BlueEloCurrent'] = test_fights['BlueEloCurrent'].fillna(initial_elo)
test_fights['EloDiff'] = np.abs(test_fights['RedEloCurrent'] - test_fights['BlueEloCurrent'])

In [873]:
data = pd.concat([data, test_fights])

In [874]:
data = data.drop(columns=['Winner'])

In [875]:
from sklearn.preprocessing import QuantileTransformer

colunas_a_normalizar = [
    'BlueHeightCms', 'BlueReachCms', 'BlueWeightLbs',
    'RedHeightCms', 'RedReachCms', 'RedWeightLbs',
    'RedEloCurrent', 'BlueEloCurrent', 'EloDiff'
]

transformer = QuantileTransformer()
data[colunas_a_normalizar] = transformer.fit_transform(data[colunas_a_normalizar])


In [876]:
data = data.drop(columns=['_merge'])

In [877]:
object_cols = data.select_dtypes(include=['object']).columns.difference(['RedFighter', 'BlueFighter', 'Date'])


data = pd.get_dummies(data, columns=object_cols)

In [878]:


from sklearn.preprocessing import LabelEncoder

lencoder = LabelEncoder()

all_fighters = pd.concat([data['RedFighter'], data['BlueFighter']]).unique()

lencoder.fit(all_fighters)

test_fights['RedFighter'] = lencoder.transform(test_fights['RedFighter'])
test_fights['BlueFighter'] = lencoder.transform(test_fights['BlueFighter'])

data['RedFighter'] = lencoder.transform(data['RedFighter'])
data['BlueFighter'] = lencoder.transform(data['BlueFighter'])





In [879]:
test_fights= data.merge(
    test_fights[['RedFighter', 'BlueFighter', 'Date']],
    on=['RedFighter', 'BlueFighter','Date'],
    how='inner'
)

In [880]:
test_fights_tratados = test_fights.drop(columns=['Date'])

In [881]:
dados_treinamento = pd.read_csv('data/dados_tratados.csv')
dados_treinamento = dados_treinamento.drop(columns=['Winner'])
test_fights_tratados = test_fights_tratados[dados_treinamento.columns]

In [882]:
raw_data = pd.read_csv('data/upcoming.csv')

## Predicao em lutas sem resultado

In [883]:
import os
import joblib
import glob

def carregar_modelos(diretorio='models'):
    modelos = {}
    caminho_padro = os.path.join(diretorio, '*.pkl')
    
    for arquivo in glob.glob(caminho_padro):
        nome_modelo = os.path.splitext(os.path.basename(arquivo))[0]
        modelo = joblib.load(arquivo)  
        modelos[nome_modelo] = modelo
        print(f"Modelo '{nome_modelo}' carregado com sucesso!")
    
    return modelos

modelos_carregados = carregar_modelos()


Modelo 'xgboost' carregado com sucesso!
Modelo 'random_forest' carregado com sucesso!
Modelo 'mlp' carregado com sucesso!
Modelo 'logistic_regression' carregado com sucesso!


Como o modelo com a melhor acuracia media ao utilizar o cross-validation foi a Regressao Logistica, sera ela que iremos utilizar para as predicoes

### Predicao de Lutas

In [884]:
df_resultadoos = pd.DataFrame()

* Regressao Logistica

In [885]:
for i in range(len(raw_data)):
    X = dados_treinamento.iloc[[i], :].values  
    resultados = modelos_carregados['logistic_regression'].predict_proba(X)

    print(f"Luta {i + 1} - {raw_data['RedFighter'][i]} vs {raw_data['BlueFighter'][i]}")
    print(f"Probabilidade do vencedor ser {raw_data['RedFighter'][i]}: {resultados[0][1] * 100:.2f}%")
    print(f"Probabilidade do vencedor ser {raw_data['BlueFighter'][i]}: {resultados[0][0] * 100:.2f}%\n")


Luta 1 - Marvin Vettori vs Roman Dolidze
Probabilidade do vencedor ser Marvin Vettori: 97.45%
Probabilidade do vencedor ser Roman Dolidze: 2.55%

Luta 2 - Chidi Njokuani vs Elizeu Zaleski dos Santos
Probabilidade do vencedor ser Chidi Njokuani: 95.05%
Probabilidade do vencedor ser Elizeu Zaleski dos Santos: 4.95%

Luta 3 - Alexander Hernandez vs Kurt Holobaugh
Probabilidade do vencedor ser Alexander Hernandez: 100.00%
Probabilidade do vencedor ser Kurt Holobaugh: 0.00%

Luta 4 - Da'Mon Blackshear vs Cody Gibson
Probabilidade do vencedor ser Da'Mon Blackshear: 98.50%
Probabilidade do vencedor ser Cody Gibson: 1.50%

Luta 5 - Diyar Nurgozhay vs Brendson Ribeiro
Probabilidade do vencedor ser Diyar Nurgozhay: 94.71%
Probabilidade do vencedor ser Brendson Ribeiro: 5.29%

Luta 6 - SeungWoo Choi vs Kevin Vallejos
Probabilidade do vencedor ser SeungWoo Choi: 93.42%
Probabilidade do vencedor ser Kevin Vallejos: 6.58%

Luta 7 - Waldo Cortes-Acosta vs Ryan Spann
Probabilidade do vencedor ser Wald

* MLP

In [887]:
for i in range(len(raw_data)):
    X = dados_treinamento.iloc[[i], :].values  
    resultados = modelos_carregados['mlp'].predict_proba(X)

    print(f"Luta {i + 1} - {raw_data['RedFighter'][i]} vs {raw_data['BlueFighter'][i]}")
    print(f"Probabilidade do vencedor ser {raw_data['RedFighter'][i]}: {resultados[0][1] * 100:.2f}%")
    print(f"Probabilidade do vencedor ser {raw_data['BlueFighter'][i]}: {resultados[0][0] * 100:.2f}%\n")

Luta 1 - Marvin Vettori vs Roman Dolidze
Probabilidade do vencedor ser Marvin Vettori: 76.04%
Probabilidade do vencedor ser Roman Dolidze: 23.96%

Luta 2 - Chidi Njokuani vs Elizeu Zaleski dos Santos
Probabilidade do vencedor ser Chidi Njokuani: 83.66%
Probabilidade do vencedor ser Elizeu Zaleski dos Santos: 16.34%

Luta 3 - Alexander Hernandez vs Kurt Holobaugh
Probabilidade do vencedor ser Alexander Hernandez: 81.42%
Probabilidade do vencedor ser Kurt Holobaugh: 18.58%

Luta 4 - Da'Mon Blackshear vs Cody Gibson
Probabilidade do vencedor ser Da'Mon Blackshear: 87.14%
Probabilidade do vencedor ser Cody Gibson: 12.86%

Luta 5 - Diyar Nurgozhay vs Brendson Ribeiro
Probabilidade do vencedor ser Diyar Nurgozhay: 77.87%
Probabilidade do vencedor ser Brendson Ribeiro: 22.13%

Luta 6 - SeungWoo Choi vs Kevin Vallejos
Probabilidade do vencedor ser SeungWoo Choi: 87.10%
Probabilidade do vencedor ser Kevin Vallejos: 12.90%

Luta 7 - Waldo Cortes-Acosta vs Ryan Spann
Probabilidade do vencedor ser

# Análise do Modelo de Previsão para Apostas no MMA

## Resultados Obtidos Apos dia 15/03

Após testar o modelo neste pequeno conjunto de dados, obtivemos uma **acurácia média de ...%**. Apesar da queda na acurácia em comparação com os resultados obtidos durante o **cross-validation**, isso pode estar relacionado ao tamanho reduzido do conjunto de testes, que pode sofrer com a **variabilidade devido ao número limitado de amostras**. Conjuntos menores são mais suscetíveis a flutuações estatísticas, o que pode distorcer a avaliação do desempenho real do modelo.

Mesmo assim, o modelo se mostra **mais eficaz** do que simplesmente seguir a **odd mais baixa** (considerada a mais segura) para se apostar, uma técnica que tem uma eficácia média de cerca de **65%** das ocasiões, conforme observado em análises históricas.

---

## A Importância de Machine Learning no MMA

Em suma, apesar da **imprevisibilidade presente no mundo do MMA**, onde fatores como lesões, imprevistos durante as lutas e a volatilidade dos atletas muitas vezes contradizem as probabilidades estimadas, o uso de técnicas de **machine learning** permite trazer um pouco de **previsibilidade e embasamento científico** a este ambiente caótico.

A aplicação de modelos preditivos não apenas supera estratégias simplistas, como seguir as odds mais baixas, mas também abre caminho para análises mais sofisticadas, considerando variáveis como:

- **Estilo de luta** (grappler vs. striker)
- **Histórico de desempenho** dos atletas

---

## Limitações e Considerações Finais

No entanto, é importante ressaltar que **nenhum modelo é infalível**, especialmente em um domínio tão dinâmico quanto o MMA. A combinação de:

1. **Dados históricos**
2. **Análise estatística**
3. **Entendimento contextual** das lutas

pode maximizar a eficácia das previsões, mas a **incerteza** sempre será uma parte intrínseca desse esporte. Portanto, o uso de machine learning deve ser visto como uma **ferramenta complementar**, capaz de auxiliar na tomada de decisões, mas não como uma garantia absoluta de sucesso.

---